In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def load_coordinates(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        next(reader)
        return np.array([[float(row[1]), float(row[2])] for row in reader])
        

In [ ]:
def load_distances(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        distances = [[float(value) for value in row] for row in reader]
    return np.array(distances)

In [ ]:
def greedy_tsp(distances,qtd_matrix):
    num_nodes = len(distances)
    visited = [False] * num_nodes
    path = [0]
    visited[0] = True
    dist =0
    time = 0

    for _ in range(1, num_nodes):
        last = path[-1]
        next_node = min(
            [(i, dist) for i, dist in enumerate(distances[last]) if not visited[i]],
            key=lambda x: x[1]
        )[0]
        dist+= distances[last][next_node]
        time += distances[last][next_node]/5 +(2/60)*qtd_matrix[last]

        visited[next_node] = True
        path.append(next_node)
   
    return {'sequence':path,'distance':dist,'time':time}

# Teste TSP greedy

###### Nesta seção testamos o algoritmo greedy para todos os clusters criados. Separamos os dados por leiturista e executamos um tsp para cada um dos 22 dias.

In [ ]:
data = pd.read_csv('../clusterizacao/dados_tratados_clusterizados.csv')
data.columns

In [ ]:
from haversine import haversine, Unit
data = pd.read_csv('../clusterizacao/dados_tratados_clusterizados.csv')
grouped=data.groupby('cluster_leiturista')
solutions =[]
for _, group in grouped:
   
    clusters = group.groupby('cluster_dia')
    solution_leiturista = []
    solution_distance = 0

    for label, cluster in clusters:
        coords = cluster[['LATITUDE', 'LONGITUDE']].to_numpy()
        n = len(coords)
        distance_matrix = np.zeros((n, n))
        qtd_matrix = cluster['QUANTIDADE_HIDROMETROS'].values
        qtd_matrix=np.insert(qtd_matrix,[0],[0],axis=0)
        for i in range(n):
            for j in range(i + 1, n):
                distance_matrix[i, j] = distance_matrix[j, i] = haversine(coords[i], coords[j], unit=Unit.KILOMETERS)

        solution = greedy_tsp(distances=distance_matrix,qtd_matrix=qtd_matrix)
        solution_leiturista.append({f"sequence_dia_{cluster['cluster_dia'].values[0]}":solution['sequence'],f"distance_dia_{cluster['cluster_dia'].values[0]}":solution['distance'],f"tempo_dia_{cluster['cluster_dia'].values[0]}":solution['time']})
    solutions.append({f"leiturista{group['cluster_leiturista'].values[0]}":solution_leiturista})

pd.DataFrame(solutions).to_json('./tsp_greedy_teste_AMOSTRA_TOTAL.json',indent=2)

In [ ]:
import json
import matplotlib.pyplot as plt

# Carrega os dados originais para obter as coordenadas
data = pd.read_csv('../../clusterizacao/dados_tratados_clusterizados_10_10.csv')

# Carrega o arquivo JSON com os resultados do TSP
with open('./tsp_greedy_teste_AMOSTRA_TOTAL2.json') as file:
    tsp_results = json.load(file)

# Preparar a plotagem
fig, ax = plt.subplots(figsize=(12, 8))

# Gerar uma paleta de cores baseada no número de leituristas
color_map = plt.cm.get_cmap('viridis', len(tsp_results))

for idx, (leiturista, days) in enumerate(tsp_results.items()):
    for day_data in days.values():
        for route in day_data:
            # Extrai os índices das sequências visitadas
            sequence_idx = route['sequence_dia_10']
            # Recupera as coordenadas
            coords = data.loc[sequence_idx, ['LATITUDE', 'LONGITUDE']].values

            # Plotar a rota
            ax.plot(coords[:, 1], coords[:, 0], marker='o', linestyle='-', color=color_map(idx), label=f'{leiturista}, Dia 10')
            ax.text(coords[0, 1], coords[0, 0], 'Start', color=color_map(idx))
            ax.text(coords[-1, 1], coords[-1, 0], 'End', color=color_map(idx))

ax.set_title('Visualização das Rotas TSP')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
plt.show()


In [ ]:
# distance_file_path = '.\clustered_data\geohash_75cjt_cluster_0_distances.csv'
# distances = load_distances(distance_file_path)
# optimal_path = greedy_tsp(distances)
# coordinates = load_coordinates('.\clustered_data\geohash_75cjt_cluster_0.csv')

In [ ]:
# print("Optimal Path:", optimal_path)

In [ ]:
def visualize_path_geo(lat_lon_coords, path):
    latitudes = [lat_lon_coords[i][0] for i in path]
    longitudes = [lat_lon_coords[i][1] for i in path]
    
    latitudes.append(lat_lon_coords[path[0]][0])
    longitudes.append(lat_lon_coords[path[0]][1])

    plt.figure(figsize=(10, 6))
    plt.plot(longitudes, latitudes, marker='o')
    for index, (lat, lon) in enumerate(lat_lon_coords):
        plt.text(lon, lat, str(index), fontsize=12)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Traveling Salesman Path Visualization (Geographic Coordinates)')
    plt.grid(True)
    plt.show()

lat_lon_coords = coordinates

path = optimal_path

visualize_path_geo(lat_lon_coords, path)